# Comparison between GraphRNN and GRAN

## Setup

In [4]:
import os
import sys
import torch
import logging
import traceback
import numpy as np
from pprint import pprint
import pandas as pd
from runner.train_runners import *
from utils.logger import setup_logging
from utils.arg_helper import parse_arguments, get_config
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
torch.set_printoptions(profile='full')


## Useful Functions

In [5]:
df = pd.read_csv("save_model_learning.csv")
df

,Unnamed: 0,Date,dataset_name,dataset_num,model_name,num_epochs,file_dir
0,0,2021-Aug-15-01-56-10,community2,500,GRANMixtureBernoulli,5,exp/GRAN/community2\GRANMixtureBernoulli_commu...
1,0,2021-Aug-15-02-09-00,community4,500,GRANMixtureBernoulli,5,exp/GRAN/community4\GRANMixtureBernoulli_commu...
2,0,2021-Aug-15-02-29-46,community8,500,GRANMixtureBernoulli,5,exp/GRAN/community8\GRANMixtureBernoulli_commu...
3,0,2021-Aug-16-14-28-27,watts,500,GRANMixtureBernoulli,5,exp/GRAN/watts\GRANMixtureBernoulli_watts_2021...
4,0,2021-Aug-17-16-56-29,barabasi,500,GRANMixtureBernoulli,15,exp/GRAN/barabasi\GRANMixtureBernoulli_barabas...
5,0,2021-Aug-17-22-00-39,barabasi,500,GRANMixtureBernoulli,50,exp/GRAN/barabasi\GRANMixtureBernoulli_barabas...
6,0,2021-Aug-18-13-17-52,community2,500,RNN,1000,exp/GraphRNN/rnn/community2\RNN_community2_202...
7,0,2021-Aug-18-16-05-01,community4,500,RNN,1000,exp/GraphRNN/rnn/community4\RNN_community4_202...
8,0,2021-Aug-18-17-43-34,community4,500,RNN,1000,exp/GraphRNN/rnn/community4\RNN_community4_202...
9,0,2021-Aug-18-22-08-24,community8,500,RNN,1000,exp/GraphRNN/rnn/community8\RNN_community8_202...


## Research Questions
1) Which one of the model is the better for each dataset?

2) About GRAN, which node ordering gives better results ?

3) Is there a bias from GRAN to generates communities ? ( How many ? Scalability ? Robustness ?)

4) What are optimal M-parametes for GraphRNN for each dataset ?
Does the parameter tweaking change the efficiency significantly ?

5) Does the SotA autoregressive model are able to retain the small-world propertie from the graph ( = avg length btw 2 nodes is proportional to the log of the number of nodes N )
(create GNN classifier "small-world" or GAN)

## Experiments
####  1) Which one of the model is the better for each dataset?
####  2) About GRAN, which node ordering gives better results ?
####  3) Is there a bias from GRAN to generates communities ? ( How many ? Scalability ? Robustness ?)
####  4) What are optimal M-parametes for GraphRNN for each dataset ? Does the parameter tweaking change the efficiency significantly

## Datasets

### Erdos Renyi dataset
Parameters used : 500 graphs btw 100-200 nodes with p=0.1

Node ordering (GRAN) :

### Barabasi Albert Dataset
Parameters used :Parameters used : 500 graphs btw 100-200 nodes with k=4/5

Node ordering (GRAN) :

### Watts Strogatz Dataset
Parameters used :Parameters used : 500 graphs btw 100-200 nodes with p=0.01

Node ordering (GRAN) : DFS


### Community Dataset
Parameters used : graphs of 2/4/8 communities of between 12 to 17 nodes

Node ordering (GRAN) : DFS


## Results

In [6]:
def get_stats_from_trained_model(config,seed):
    """Return all mmd statistical results from
    generated graph by the trained model, in the form of a dict"""

    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    config.use_gpu = config.use_gpu and torch.cuda.is_available()
    torch.cuda.empty_cache()

    runner = eval(config.runner)(config)

    mmd_degree_test, mmd_clustering_test, mmd_4orbits_test, mmd_spectral_test = runner.test()
    return {"mmd_degree_test": mmd_degree_test, "mmd_clustering_test": mmd_clustering_test,
            "mmd_4orbits_test": mmd_4orbits_test, "mmd_spectral_test": mmd_spectral_test}

max # nodes = 32 || mean # nodes = 32.0
max # edges = 170 || mean # edges = 162.574


100%|██████████| 10/10 [00:28<00:00,  2.81s/it]


max # nodes = 32 || mean # nodes = 32.0
max # edges = 170 || mean # edges = 162.574


100%|██████████| 10/10 [00:27<00:00,  2.77s/it]


max # nodes = 32 || mean # nodes = 32.0
max # edges = 170 || mean # edges = 162.574


...

100%|██████████| 20/20 [05:50<00:00, 17.52s/it]


max # nodes = 199 || mean # nodes = 149.5
max # edges = 199 || mean # edges = 149.5


100%|██████████| 20/20 [05:43<00:00, 17.18s/it]


max # nodes = 199 || mean # nodes = 149.5
max # edges = 199 || mean # edges = 149.5


100%|██████████| 20/20 [05:47<00:00, 17.36s/it]


In [47]:
row_list = []
for training_path in df['file_dir']:

    try :
        config_path = os.path.join(training_path, 'config.yaml')
        config = get_config(config_path)
    except :
        continue

    for i in range(10):
        if training_path.find('mlp') == -1:
            dict_results = {"dataset_name": config.dataset.name, "model_name": config.model.name,
                    "num_epochs": config.train.max_epoch}
        else :
            dict_results = {"dataset_name": config.dataset.name, "model_name": config.model.name+"_MLP",
                    "num_epochs": config.train.max_epoch}
        dict_stats = get_stats_from_trained_model(config,11*(i^3))
        dict_results.update(dict_stats)
        row_list.append(dict_results)
        torch.cuda.empty_cache()

result_df=pd.DataFrame(row_list)
torch.cuda.empty_cache()

TypeError: get_stats_from_trained_model() takes 1 positional argument but 2 were given

In [7]:
result_df
result_df.to_csv("statsResults.csv")

,dataset_name,model_name,num_epochs,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
0,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
1,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
2,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
3,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
4,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
...,...,...,...,...,...,...,...
195,watts_ring,GRANMixtureBernoulli,50,0.001969,0.000000,0.000016,0.039995
196,watts_ring,GRANMixtureBernoulli,50,0.001969,0.000000,0.000016,0.039995
197,watts_ring,GRANMixtureBernoulli,50,0.001969,0.000000,0.000016,0.039995
198,watts_ring,GRANMixtureBernoulli,50,0.001969,0.000000,0.000016,0.039995


In [8]:
result_df.style

,dataset_name,model_name,num_epochs,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
0,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
1,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
2,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
3,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
4,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
5,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
6,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
7,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
8,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
9,community2,GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724


Here are the mmd (with emd) metrics results for each data
#### 2-Community Dataset stats

In [31]:
result_df[result_df['dataset_name']=="community2"].groupby(['model_name', 'num_epochs']).mean().style.highlight_min(color = 'lightblue', axis = 0)

,,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
model_name,num_epochs,,,,
GRANMixtureBernoulli,5,0.017374,0.220196,0.139863,0.072724
RNN,1000,0.033613,0.250731,0.369406,0.121066
RNN_MLP,1000,0.051372,0.250667,0.493045,0.133577


#### 4-Community Dataset stats

In [32]:
result_df[result_df['dataset_name']=="community4"].groupby(['model_name', 'num_epochs']).mean().style.highlight_min(color = 'lightblue', axis = 0)

,,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
model_name,num_epochs,,,,
GRANMixtureBernoulli,5,0.008479,0.027914,0.455951,0.018261
RNN,1000,0.022312,0.057453,0.573198,0.027762
RNN_MLP,1000,0.051044,0.057449,0.571152,0.039798


#### 8-Community Dataset stats

In [33]:
result_df[result_df['dataset_name']=="community8"].groupby(['model_name', 'num_epochs']).mean().style.highlight_min(color = 'lightblue', axis = 0)

,,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
model_name,num_epochs,,,,
GRANMixtureBernoulli,5,0.017635,0.014175,0.518267,0.004593
RNN,1000,0.062131,0.048314,0.665375,0.058272
RNN_MLP,1000,0.102437,0.051067,0.702279,0.066848


#### Barabasi Dataset stats

In [34]:
result_df[result_df['dataset_name']=="barabasi"].groupby(['model_name', 'num_epochs']).mean().style.highlight_min(color = 'lightblue', axis = 0)

#### Watts-Strogatz (p=0.05 Graph mode) Dataset stats

In [35]:
result_df[result_df['dataset_name']=="watts"].groupby(['model_name', 'num_epochs']).mean().style.highlight_min(color = 'lightblue', axis = 0)


,,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
model_name,num_epochs,,,,
GRANMixtureBernoulli,5,0.121858,0.029865,0.001750,0.179487
RNN,1000,0.000193,0.000014,0.000005,0.011747
RNN_MLP,1000,0.000074,0.000020,0.000000,0.009267


#### Watts-Strogatz (Ring p =0.0 Graph mode) Dataset stats

In [36]:
result_df[result_df['dataset_name']=="watts_ring"].groupby(['model_name', 'num_epochs']).mean().style.highlight_min(color = 'lightblue', axis = 0)

,,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
model_name,num_epochs,,,,
GRANMixtureBernoulli,50,0.001969,0.000000,0.000016,0.039995
RNN,1000,0.014658,0.000023,0.000025,0.075430
RNN_MLP,1000,0.005352,0.000000,0.000066,0.051669


#### Summary measurements

In [38]:
result_df.groupby(['dataset_name', 'model_name', 'num_epochs']).agg(['mean', 'std'])

mmd_degree_test            \
                                                        mean       std   
dataset_name model_name           num_epochs                             
barabasi     GRANMixtureBernoulli 15                0.049913  0.000481   
                                  50                0.030368  0.000551   
             RNN                  1000              0.143479  0.000818   
             RNN_MLP              1000              0.145420  0.002246   
community2   GRANMixtureBernoulli 5                 0.017374  0.000000   
             RNN                  1000              0.033613  0.000000   
             RNN_MLP              1000              0.051372  0.000000   
community4   GRANMixtureBernoulli 5                 0.008479  0.000001   
             RNN                  1000              0.022312  0.000000   
             RNN_MLP              1000              0.051044  0.000000   
community8   GRANMixtureBernoulli 5                 0.017635  0.000000   
             RNN                  1000              0.062131  0.000000   
             RNN_MLP              1000              0.102437  0.000000   
watts        GRANMixtureBernoulli 5                 0.121858  0.000738   
             RNN                  1000              0.000193  0.000033   
             RNN_MLP              1000              0.000074  0.000022   
watts_ring   GRANMixtureBernoulli 50                0.001969  0.000000   
             RNN                  1000              0.014658  0.000000   
             RNN_MLP              1000              0.005352  0.000000   

                                             mmd_clustering_test            \
                                                            mean       std   
dataset_name model_name           num_epochs                                 
barabasi     GRANMixtureBernoulli 15                    0.084001  0.002337   
                                  50                    0.081585  0.001243   
             RNN                  1000                  0.365482  0.002339   
             RNN_MLP              1000                  0.342212  0.032601   
community2   GRANMixtureBernoulli 5                     0.220196  0.000000   
             RNN                  1000                  0.250731  0.000000   
             RNN_MLP              1000                  0.250667  0.000000   
community4   GRANMixtureBernoulli 5                     0.027914  0.000003   
             RNN                  1000                  0.057453  0.000000   
             RNN_MLP              1000                  0.057449  0.000000   
community8   GRANMixtureBernoulli 5                     0.014175  0.000000   
             RNN                  1000                  0.048314  0.000000   
             RNN_MLP              1000                  0.051067  0.000000   
watts        GRANMixtureBernoulli 5                     0.029865  0.000502   
             RNN                  1000                  0.000014  0.000006   
             RNN_MLP              1000                  0.000020  0.000010   
watts_ring   GRANMixtureBernoulli 50                    0.000000  0.000000   
             RNN                  1000                  0.000023  0.000000   
             RNN_MLP              1000                  0.000000  0.000000   

                                             mmd_4orbits_test                \
                                                         mean           std   
dataset_name model_name           num_epochs                                  
barabasi     GRANMixtureBernoulli 15             3.020837e-02  9.096069e-04   
                                  50             2.848619e-02  1.258360e-03   
             RNN                  1000           1.807474e-01  1.117480e-03   
             RNN_MLP              1000           1.798068e-01  2.338711e-02   
community2   GRANMixtureBernoulli 5              1.398629e-01  0.000000e+00   
             RNN                  1000           3.694064e-01  0.00

Testing repeated measurements with the different seed ?

In [48]:
row_list = []
for training_path in df['file_dir']:

    try :
        config_path = os.path.join(training_path, 'config.yaml')
        config = get_config(config_path)
    except :
        continue
    if config.dataset.name == "community4" and config.model.name == "RNN":
        for i in range(10):
            if training_path.find('mlp') == -1:
                dict_results = {"dataset_name": config.dataset.name, "model_name": config.model.name,
                        "num_epochs": config.train.max_epoch}
            else :
                dict_results = {"dataset_name": config.dataset.name, "model_name": config.model.name+"_MLP",
                        "num_epochs": config.train.max_epoch}
            dict_stats = get_stats_from_trained_model(config)
            dict_results.update(dict_stats)
            row_list.append(dict_results)
            torch.cuda.empty_cache()

result_community_df=pd.DataFrame(row_list)
torch.cuda.empty_cache()

max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038


D:\anaconda3\envs\graphRL\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 335.038
max # nodes = 64 || mean # nodes = 64.0
max # edges = 347 || mean # edges = 

In [46]:
result_community_df


,dataset_name,model_name,num_epochs,mmd_degree_test,mmd_clustering_test,mmd_4orbits_test,mmd_spectral_test
0,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
1,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
2,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
3,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
4,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
5,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
6,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
7,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
8,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762
9,community4,RNN,1000,0.022312,0.057453,0.573198,0.027762


In [49]:
result_community_df.groupby(['dataset_name', 'model_name', 'num_epochs']).agg(['mean', 'std'])



mmd_degree_test      mmd_clustering_test  \
                                              mean  std                mean   
dataset_name model_name num_epochs                                            
community4   RNN        1000              0.022312  0.0            0.057453   
             RNN_MLP    1000              0.051044  0.0            0.057449   

                                        mmd_4orbits_test       \
                                    std             mean  std   
dataset_name model_name num_epochs                              
community4   RNN        1000        0.0         0.573198  0.0   
             RNN_MLP    1000        0.0         0.571152  0.0   

                                   mmd_spectral_test       
                                                mean  std  
dataset_name model_name num_epochs                         
community4   RNN        1000                0.027762  0.0  
             RNN_MLP    1000                0.039798  0.0

In [51]:
result_df.groupby(['dataset_name', 'model_name', 'num_epochs']).describe()


mmd_degree_test            \
                                                       count      mean   
dataset_name model_name           num_epochs                             
barabasi     GRANMixtureBernoulli 15                    10.0  0.049913   
                                  50                    10.0  0.030368   
             RNN                  1000                  10.0  0.143479   
             RNN_MLP              1000                  20.0  0.145420   
community2   GRANMixtureBernoulli 5                     10.0  0.017374   
             RNN                  1000                  10.0  0.033613   
             RNN_MLP              1000                  10.0  0.051372   
community4   GRANMixtureBernoulli 5                     10.0  0.008479   
             RNN                  1000                  10.0  0.022312   
             RNN_MLP              1000                  10.0  0.051044   
community8   GRANMixtureBernoulli 5                     10.0  0.017635   
             RNN                  1000                  10.0  0.062131   
             RNN_MLP              1000                  10.0  0.102437   
watts        GRANMixtureBernoulli 5                     10.0  0.121858   
             RNN                  1000                  10.0  0.000193   
             RNN_MLP              1000                  10.0  0.000074   
watts_ring   GRANMixtureBernoulli 50                    10.0  0.001969   
             RNN                  1000                  10.0  0.014658   
             RNN_MLP              1000                  10.0  0.005352   

                                                                            \
                                                   std       min       25%   
dataset_name model_name           num_epochs                                 
barabasi     GRANMixtureBernoulli 15          0.000481  0.049240  0.049667   
                                  50          0.000551  0.029278  0.030204   
             RNN                  1000        0.000818  0.141956  0.143162   
             RNN_MLP              1000        0.002246  0.142889  0.143220   
community2   GRANMixtureBernoulli 5           0.000000  0.017374  0.017374   
             RNN                  1000        0.000000  0.033613  0.033613   
             RNN_MLP              1000        0.000000  0.051372  0.051372   
community4   GRANMixtureBernoulli 5           0.000001  0.008477  0.008477   
             RNN                  1000        0.000000  0.022312  0.022312   
             RNN_MLP              1000        0.000000  0.051044  0.051044   
community8   GRANMixtureBernoulli 5           0.000000  0.017635  0.017635   
             RNN                  1000        0.000000  0.062131  0.062131   
             RNN_MLP              1000        0.000000  0.102437  0.102437   
watts        GRANMixtureBernoulli 5           0.000738  0.120851  0.121331   
             RNN                  1000        0.000033  0.000136  0.000180   
             RNN_MLP              1000        0.000022  0.000043  0.000059   
watts_ring   GRANMixtureBernoulli 50          0.000000  0.001969  0.001969   
             RNN                  1000        0.000000  0.014658  0.014658   
             RNN_MLP              1000        0.000000  0.005352  0.005352   

                                                                            \
                                                   50%       75%       max   
dataset_name model_name           num_epochs                                 
barabasi     GRANMixtureBernoulli 15          0.049841  0.050379  0.050537   
                                  50          0.030370  0.030650  0.031299   
             RNN                  1000        0.143391  0.143955  0.145020   
             RNN_MLP              1000        0.145417  0.147277  0.149144   
community2   GRANMixtureBernoulli 5           0.017374  0.017374  0.017374   
             RNN                  1000        0.033613  0.033613  0.033613 